In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# CIFAR-10 dataset has 10 classes
num_classes = 10

In [ ]:
# Image Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])

In [ ]:
# CIFAR-10 data
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)

100%|██████████| 170498071/170498071 [00:13<00:00, 12193890.25it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Define a GoogLeNet model for CIFAR-10
class GoogLeNetCIFAR10(models.GoogLeNet):
    def __init__(self, num_classes=10):
        super(GoogLeNetCIFAR10, self).__init__(num_classes=num_classes, aux_logits=False, init_weights=True)
        # Redefine the first convolutional layer to accept CIFAR-10's 32x32 pixel images
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Follow the forward pass of the GoogLeNet model
        return super(GoogLeNetCIFAR10, self).forward(x)


In [ ]:
# Instantiate the model and transfer it to the GPU
model = GoogLeNetCIFAR10(num_classes=num_classes).to(device)


In [ ]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Function to train the model
def train(model, criterion, optimizer, train_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            # Transfer inputs and labels to the GPU
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100}')
                running_loss = 0.0

In [ ]:
# Function to test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            # Transfer images and labels to the GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# Start training
train(model, criterion, optimizer, train_loader, epochs=10)



Epoch 1, Batch 100, Loss: 1.9158953213691712
Epoch 1, Batch 200, Loss: 1.7259334874153138
Epoch 1, Batch 300, Loss: 1.5547630107402801
Epoch 1, Batch 400, Loss: 1.4671156442165374
Epoch 1, Batch 500, Loss: 1.3805170524120332
Epoch 1, Batch 600, Loss: 1.293321840763092
Epoch 1, Batch 700, Loss: 1.2353383630514145
Epoch 2, Batch 100, Loss: 1.1351709514856339
Epoch 2, Batch 200, Loss: 1.0613451540470122
Epoch 2, Batch 300, Loss: 1.0482701855897902
Epoch 2, Batch 400, Loss: 0.9935551083087921
Epoch 2, Batch 500, Loss: 0.9518651217222214
Epoch 2, Batch 600, Loss: 0.9359095537662506
Epoch 2, Batch 700, Loss: 0.8961749660968781
Epoch 3, Batch 100, Loss: 0.831555083990097
Epoch 3, Batch 200, Loss: 0.7953695210814477
Epoch 3, Batch 300, Loss: 0.772147698700428
Epoch 3, Batch 400, Loss: 0.7850076878070831
Epoch 3, Batch 500, Loss: 0.7473026326298714
Epoch 3, Batch 600, Loss: 0.7388499349355697
Epoch 3, Batch 700, Loss: 0.6884658551216125
Epoch 4, Batch 100, Loss: 0.6176674127578735
Epoch 4, Batc

In [ ]:
# Test the model
accuracy = test(model, test_loader)
print(f'Accuracy of the model on the 10000 test images: {accuracy}%')


Accuracy of the model on the 10000 test images: 82.72%
